### Emma Klemets, 260775167   
All my work, code, output and written answers are in this notebook.

In [1]:
#for interactive plots - not what I want

# %matplotlib
# %matplotlib inline
# %matplotlib notebook

In [157]:
import numpy as np
import matplotlib

#for pop out plots - windows
matplotlib.use('Qt5Agg')
import matplotlib.pyplot as plt
import time
import matplotlib.cm as cm
from mpl_toolkits.mplot3d import Axes3D

Make a 3-D nbody code that calculates the forces by computing the potential, where the potential is found by convolving the density with the (softened) potential from a single particle. The acceleration is then found by taking the gradient of the potential. You will probably wish to use a leapfrog solver with fixed timestep.

In [3]:
"""
calculates the forces by computing the potential

potential is found by convolving the density with the (softened) potential from a single particle. 
phi = rho x V_1

The acceleration is then found by taking the gradient of the potential.
acc = grad phi


calculate the potential using the Green's function!
So you can use the Green's function of the laplacian : -1/(4*pi*r) where r = sqrt(x^2 + y^2 + z^2)
Since (taking 4*pi*G=1): laplacian(phi) = rho ,

solve phi = ifft(fft(G_laplacian) * fft(rho))  where G_laplacian is just the Green's function of the laplacian

- how does it not matter which particle for rho?
- what is the r in G's relative to - should it be between two particles? 
"""
x=1

In [172]:
class Particles:
    def __init__(self,x,v,m, gridSize, grid_dx=1, BC_per=False):
        #set up our grid - gridSize should be [x, y, z]
        x_g = np.arange(0, gridSize[0], grid_dx)
        y_g = np.arange(0, gridSize[1], grid_dx)
        z_g = np.arange(0, gridSize[2], grid_dx)
        self.grid = np.meshgrid(x_g, y_g, z_g)
        self.grid_shape = np.array(gridSize)
        self.dt = grid_dx
        self.f_grid = np.meshgrid(self.grid[0][0]*0, self.grid[0][0]*0, self.grid[0][0]*0)
        
        self.BC = BC_per #true for periodic BC, false for not
        
        self.x=x.copy()
        self.v=v.copy()
        try:
            self.m=m.copy()
        except:
            self.m=m
            
        self.f=np.empty(x.shape)
        self.a=np.empty(x.shape)
        self.n=self.x.shape[0]
        
        #maybe save phi / energy?
        
    def r(self):
        return np.sqrt(np.sum(self.x**2, axis=1))
        
    def get_forces(self, soft=0.01, do_pot=False):
        self.f[:]=0
        for i in range(self.n):
            for j in range(self.n):
                if i!=j:
                    dx=-self.x[i,:]+self.x[j,:]
                    drsqr=np.sum(dx**2)
                    
                    if drsqr<soft**2:
                        drsqr = soft**2
                
                    r3=1/(drsqr*np.sqrt(drsqr))
                    
                    self.f[i,:]=self.f[i,:]+dx*self.m[j]*r3
        
    #this is for calculting forces with position values (xx) that are not ever saved
    def get_forces_2(self, xx, soft=0.01, do_pot=False):
        self.f[:]=0
        for i in range(self.n):
            for j in range(self.n):
                if i!=j:
                    dx=-xx[i,:]+xx[j,:]
                    drsqr=np.sum(dx**2)
                    
                    if drsqr<soft**2:
                        drsqr = soft**2
                
                    r3=1/(drsqr*np.sqrt(drsqr))
                    self.f[i,:]=self.f[i,:]+dx*self.m[j]*r3

    def get_acc(self, soft=0.01, do_pot=False):
        self.a[:]=0
        for i in range(self.n):
            for j in range(self.n):
                if i!=j:
                    dx=-self.x[i,:]+self.x[j,:]
                    drsqr=np.sum(dx**2)
                    
                    if drsqr<soft**2:
                        drsqr = soft**2
                
                    r3=1/(drsqr*np.sqrt(drsqr))
                    
                    self.a[i,:]=self.a[i,:]+dx*r3 #going to be the same really as F as m=1 rn


    def take_step_leapfrog(self, dt, soft=0.01):
        self.get_acc()
        v_h = self.v + self.a*dt
        self.x = self.x + v_h*dt
        self.v = v_h #so the V saved will be a half step behind x
        

    """
    acc = grad phi

    calculate the potential using the Green's function!
    So you can use the Green's function of the laplacian : -1/(4*pi*r) where r = sqrt(x^2 + y^2 + z^2)
    Since (taking 4*pi*G=1): laplacian(phi) = rho ,

    solve phi = ifft(fft(G_laplacian) * fft(rho))  where G_laplacian is just the Green's function of the laplacian
    """
    
    def r_grid_from_point(self, point):
        t = np.zeros( (len(self.grid[0]), len(self.grid[1]), len(self.grid[2])) ) 

#         for i in range(gridSize[0]): #this is breaking for non dt=1
#             for j in range(gridSize[1]):
#                 for k in range(gridSize[2]):
#                     r = np.sqrt((self.grid[0][i][j][k] + self.dt/2 - point[0])**2 + \
#                                 (self.grid[1][i][j][k]+ self.dt/2-point[1])**2 + (self.grid[2][i][j][k]+ self.dt/2-point[2])**2)
#                     t[i][j][k] = r
            
        # could also just do this, but I want my point in the center so i don't lose most of it cause that seems 
        # problematic - well this works as long as the box is a cube
        grid_arr = np.array(self.grid)
        t = np.sum((grid_arr + (self.dt)/2.0 -point[0])**2,axis=0)

        return t
    
    def G_laplacian(self, soften=0.01):
        #can we just use a non-real particle at the center to make this easier? Yes i think so
        
        centerPoint = [self.grid_shape[0]/2.0, self.grid_shape[1]/2.0, self.grid_shape[2]/2.0]
        
        R_arr = self.r_grid_from_point(centerPoint)
    
        R_arr[R_arr<soften] = soften
        
        return -1/(4*np.pi*R_arr)
    
    def get_density(self):
        
        #use a histogram to do way easier
        
        #get our bin edges - need one extra point compared to our grid as we need all the edges
        dt = self.dt
        x_g = np.arange(0, len(self.grid[0])*dt + dt, dt) #these can't deal with particles in -ve space -> BC will deal with this?
        y_g = np.arange(0, len(self.grid[1])*dt + dt, dt)
        z_g = np.arange(0, len(self.grid[2])*dt + dt, dt)
        
        #now fill the histgram, with the masses as the weights
        Hist, edges = np.histogramdd(self.x, bins=[x_g,y_g,y_g], weights=self.m)
            
        #get total mass per grid cube, then just divide by the volume of each cube
        density = Hist/dt**3
        return density
     
    def get_forces_conv(self, soft=0.01, do_pot=False):
        #    self.f[:]=0
        #what is rho??  - the single particle potential? 
        # no I think it's the density - and somehow G's is the potential?
        rho = self.get_density() #I think this looks decent
    
        green = self.G_laplacian()
#         phi = convFunction(green, rho, p=5)  
        phi = convFunction2(green, rho, dim=len(self.x[0]))  
        
        
#         really f = grad phi
        #grad - needs BC applied
        #just df/dx = (f(x+dt)-f(x-dt))/2dt for now
        grad_x = (np.roll(phi,1,axis=0)-np.roll(phi,-1,axis=0))/(2*self.dt)
        grad_y = (np.roll(phi,1,axis=1)-np.roll(phi,-1,axis=1))/(2*self.dt)
        grad_z = (np.roll(phi,1,axis=2)-np.roll(phi,-1,axis=2))/(2*self.dt) 
#         print(len(grad_x))

        print(grad_x.shape)
        grad_x = -green*rho*grad_x
        grad_y = -green*rho*grad_y
        grad_z = -green*rho*grad_z
    
        #Apply the force to each particle
#         F_ptcls = np.moveaxis(F_gp,[0,1,2],[-1,-2,-3])

        #should stack all the forces so this is 4d -> [f_x, f_y, f_z] for each grid cell
        self.f_grid = np.stack((grad_x, grad_y, grad_z), axis=3) 
        
        return phi, self.f_grid, grad_x, grad_y, grad_z


    def get_acc_conv(self, soft=0.01, do_pot=False):
        #first calculate the force in each grid cell
        self.get_forces_conv()
        
        for i in range(self.n):
            #need to figure out which cell each particle is in
            #then pick out the force in that cell to use here
            
#             print("position? ", self.x[i,:], (self.dt))
            
            cell_n = np.floor(self.x[i,:]/(self.dt)).astype('int64') 
            #got to watch out when particles leave our cube and deal with them
            
            f = self.f_grid[cell_n[0]][cell_n[1]][cell_n[2]]
            
            print("f, cell#:", f, cell_n)
            
            self.a[i,:]=f/self.m[i]
            
        return self.a

    def take_step_leapfrog_conv(self, dt, soft=0.01):
        self.get_acc_conv()
        v_h = self.v + self.a*dt
        self.x = self.x + v_h*dt
        self.v = v_h #so the V saved will be a half step behind x
        
    def get_energy(self):
        e_K = 0.5*self.m*(np.sum(self.v**2, axis=1))**2
        
        e_P = 0 #somehow use phi to get this?
        
        e_total = e_K + e_P
        return e_total
    
#helper fucntions
def convFunction(arr1, arr2, p=5): #is this actually working well? I don't think so

    arrFT1 = np.fft.fft(np.pad(arr1, [0, p]))
    arrFT2 = np.fft.fft(np.pad(arr2, [0, p]))
        
    convolved = np.fft.ifft(arrFT1 * arrFT2)/len(arr1)
    
    if p > 0:
        convolved = convolved[:-p,:-p,:-p]

    return convolved.real

def convFunction2(arr1, arr2, dim=3): #this guy works I think
 
    arrFT1 = np.fft.rfftn(arr1)
    arrFT2 = np.fft.rfftn(arr2)
    
    phi = np.fft.irfftn(arrFT1 * arrFT2, s=arr1.shape) # need to have s to get odd length arrays

    for i in range(dim):
            phi = 0.5*(np.roll(phi,1,axis=i)+phi)    
    return phi

In [165]:
#checking stuff


In [86]:
gridSize = [5, 5, 5]
grid_dx = 1

x_g = np.arange(0, gridSize[0], grid_dx)
y_g = np.arange(0, gridSize[1], grid_dx)
z_g = np.arange(0, gridSize[2], grid_dx)
grid = np.meshgrid(x_g, y_g, z_g)

t = np.zeros( gridSize)

center = [gridSize[0]/2.0, gridSize[0]/2.0, gridSize[0]/2.0]
# center = [0,0,0]

for i in range(gridSize[0]):
    for j in range(gridSize[0]):
        for k in range(gridSize[0]):
            r = np.sqrt((grid[0][i][j][k] + dt/2 - center[0])**2 + (grid[1][i][j][k]+ dt/2-center[1])**2 + (grid[2][i][j][k]+ dt/2-center[2])**2)
            t[i][j][k] = r
    
# print(t)


grid2 = np.array(grid)
r = np.sum(grid2**2,axis=0)

# print(r-t)

In [175]:
n=1 #so this works for 1 particle, but only for certain dt/gridsize values 

x=np.zeros([n,3])
v=np.zeros([n,3])
x[0,0]=1.5 #p1 x
x[0,1]=1.5 #p1 y
x[0,2]=1.5 #p1 z

# x[1,0]=2.5 #p2 x

# x[1,1]=2.2 #p2 y

# v[0,1]=0.25
# v[1,1]=-0.25
# v[0,2]=0.25
# v[1,2]=-0.25

gridSize = [5, 5, 5]*2

m=np.ones(n) #mass

dt_t = 1.0#1.0
parts=Particles(x,v,m, gridSize, grid_dx=dt_t)
colors = cm.rainbow(np.linspace(0, 1, n))

dt=0.01

# fig = plt.figure()
# ax = Axes3D(fig)
# ax.scatter(parts.x[n_i][0], parts.x[n_i][1], parts.x[n_i][2], color=colors[n_i])
# plt.pause(0.01)
# plt.show()

#looking at the density to see if it makes sense
"""
d = parts.get_density()

x=np.arange(0, gridSize[0], dt_t)
yy,zz=np.meshgrid(x,x)

for i in range(0, int(gridSize[0]/dt_t)):
    plt.pcolor(yy,zz, d[i],  shading='auto')
    plt.title("x = {}".format(i*dt_t))
    plt.xlabel("z")
    plt.ylabel("y")
    
    plt.pause(2)
plt.colorbar()
print(d)
"""

phi_test, f_test, grad_x, grad_y, grad_z = parts.get_forces_conv()
g = parts.G_laplacian()

# x=np.arange(0, gridSize[0], dt_t)
# xx,yy,zz=np.meshgrid(x,x,xs)

x_g = np.arange(0, gridSize[0], dt_t)
y_g = np.arange(0, gridSize[1], dt_t)
z_g = np.arange(0, gridSize[2], dt_t)
grid = np.meshgrid(x_g, y_g, z_g)
xx,yy,zz = np.meshgrid(x_g, y_g, z_g)


# for i in range(0, int(gridSize[0]/dt_t)):
#     plt.pcolor(yy,zz, f_test[i],  shading='auto')
#     plt.title("x = {}".format(i*dt_t))
#     plt.xlabel("z")
#     plt.ylabel("y")
    
#     plt.pause(2)
# plt.colorbar()

print(grid[0].shape, grad_x.shape)

# print(grad_x)

# fig = plt.figure()
# ax = fig.gca(projection='3d')
# for n_i in range(n):
#     ax.scatter(parts.x[n_i][0], parts.x[n_i][1], parts.x[n_i][2], color=colors[n_i])
# ax.quiver(xx, yy, zz, grad_x, grad_y, grad_z)#, length=1.0)
# plt.pause(5)

x=np.arange(0, gridSize[0], dt_t)
yy,zz=np.meshgrid(x,x)

# for i in range(0, int(gridSize[0]/dt_t)):
#     plt.pcolor(yy,zz, phi_test[i],  shading='auto')
# #     plt.pcolor(yy,zz, g[i],  shading='auto')
    
#     plt.title("x = {}".format(i*dt_t))
#     plt.xlabel("z")
#     plt.ylabel("y")
    
#     plt.pause(2)
# plt.colorbar()

# acc = parts.get_acc_conv()

# print("As:", acc)

#well it does somthing rn, but I have a feeling I should have non-zero values like everywhere
# ok I do get acc , but is it right? - maybe compare with the other acc method
# prob need to make mesh finer, and choose a good size of box as I don't have BC yet


# parts.take_step_leapfrog_conv(0.01)

# """
colors = cm.rainbow(np.linspace(0, 1, n)) 

fig = plt.figure()
ax = Axes3D(fig)
        
for k in range(300):
    for l in range(20):
        parts.take_step_leapfrog_conv(dt)
    
    print('step ',k)

    #     plt.clf() #clears prev graphs
    for n_i in range(n):
        ax.scatter(parts.x[n_i][0], parts.x[n_i][1], parts.x[n_i][2], color=colors[n_i])
#     ax.set_title("frame {}".format(k))
#     ax.text2D(0.05, 0.95, "frame {}".format(k), transform=ax.transAxes)
    plt.pause(0.01)
# """   
l=8

(5, 5, 5)
(5, 5, 5) (5, 5, 5)
(5, 5, 5)
f, cell#: [-5.75187117e-19 -9.66314356e-19 -6.90224540e-20] [1 1 1]
(5, 5, 5)
f, cell#: [-5.75187117e-19 -9.66314356e-19 -6.90224540e-20] [1 1 1]
(5, 5, 5)
f, cell#: [-5.75187117e-19 -9.66314356e-19 -6.90224540e-20] [1 1 1]
(5, 5, 5)
f, cell#: [-5.75187117e-19 -9.66314356e-19 -6.90224540e-20] [1 1 1]
(5, 5, 5)
f, cell#: [-5.75187117e-19 -9.66314356e-19 -6.90224540e-20] [1 1 1]
(5, 5, 5)
f, cell#: [-5.75187117e-19 -9.66314356e-19 -6.90224540e-20] [1 1 1]
(5, 5, 5)
f, cell#: [-5.75187117e-19 -9.66314356e-19 -6.90224540e-20] [1 1 1]
(5, 5, 5)
f, cell#: [-5.75187117e-19 -9.66314356e-19 -6.90224540e-20] [1 1 1]
(5, 5, 5)
f, cell#: [-5.75187117e-19 -9.66314356e-19 -6.90224540e-20] [1 1 1]
(5, 5, 5)
f, cell#: [-5.75187117e-19 -9.66314356e-19 -6.90224540e-20] [1 1 1]
(5, 5, 5)
f, cell#: [-5.75187117e-19 -9.66314356e-19 -6.90224540e-20] [1 1 1]
(5, 5, 5)
f, cell#: [-5.75187117e-19 -9.66314356e-19 -6.90224540e-20] [1 1 1]
(5, 5, 5)
f, cell#: [-5.75187117e-

(5, 5, 5)
f, cell#: [-5.75187117e-19 -9.66314356e-19 -6.90224540e-20] [1 1 1]
(5, 5, 5)
f, cell#: [-5.75187117e-19 -9.66314356e-19 -6.90224540e-20] [1 1 1]
(5, 5, 5)
f, cell#: [-5.75187117e-19 -9.66314356e-19 -6.90224540e-20] [1 1 1]
(5, 5, 5)
f, cell#: [-5.75187117e-19 -9.66314356e-19 -6.90224540e-20] [1 1 1]
(5, 5, 5)
f, cell#: [-5.75187117e-19 -9.66314356e-19 -6.90224540e-20] [1 1 1]
(5, 5, 5)
f, cell#: [-5.75187117e-19 -9.66314356e-19 -6.90224540e-20] [1 1 1]
(5, 5, 5)
f, cell#: [-5.75187117e-19 -9.66314356e-19 -6.90224540e-20] [1 1 1]
(5, 5, 5)
f, cell#: [-5.75187117e-19 -9.66314356e-19 -6.90224540e-20] [1 1 1]
(5, 5, 5)
f, cell#: [-5.75187117e-19 -9.66314356e-19 -6.90224540e-20] [1 1 1]
(5, 5, 5)
f, cell#: [-5.75187117e-19 -9.66314356e-19 -6.90224540e-20] [1 1 1]
(5, 5, 5)
f, cell#: [-5.75187117e-19 -9.66314356e-19 -6.90224540e-20] [1 1 1]
(5, 5, 5)
f, cell#: [-5.75187117e-19 -9.66314356e-19 -6.90224540e-20] [1 1 1]
(5, 5, 5)
f, cell#: [-5.75187117e-19 -9.66314356e-19 -6.90224540

(5, 5, 5)
f, cell#: [-5.75187117e-19 -9.66314356e-19 -6.90224540e-20] [1 1 1]
(5, 5, 5)
f, cell#: [-5.75187117e-19 -9.66314356e-19 -6.90224540e-20] [1 1 1]
(5, 5, 5)
f, cell#: [-5.75187117e-19 -9.66314356e-19 -6.90224540e-20] [1 1 1]
(5, 5, 5)
f, cell#: [-5.75187117e-19 -9.66314356e-19 -6.90224540e-20] [1 1 1]
(5, 5, 5)
f, cell#: [-5.75187117e-19 -9.66314356e-19 -6.90224540e-20] [1 1 1]
(5, 5, 5)
f, cell#: [-5.75187117e-19 -9.66314356e-19 -6.90224540e-20] [1 1 1]
(5, 5, 5)
f, cell#: [-5.75187117e-19 -9.66314356e-19 -6.90224540e-20] [1 1 1]
(5, 5, 5)
f, cell#: [-5.75187117e-19 -9.66314356e-19 -6.90224540e-20] [1 1 1]
(5, 5, 5)
f, cell#: [-5.75187117e-19 -9.66314356e-19 -6.90224540e-20] [1 1 1]
(5, 5, 5)
f, cell#: [-5.75187117e-19 -9.66314356e-19 -6.90224540e-20] [1 1 1]
(5, 5, 5)
f, cell#: [-5.75187117e-19 -9.66314356e-19 -6.90224540e-20] [1 1 1]
(5, 5, 5)
f, cell#: [-5.75187117e-19 -9.66314356e-19 -6.90224540e-20] [1 1 1]
(5, 5, 5)
f, cell#: [-5.75187117e-19 -9.66314356e-19 -6.90224540

(5, 5, 5)
f, cell#: [-5.75187117e-19 -9.66314356e-19 -6.90224540e-20] [1 1 1]
(5, 5, 5)
f, cell#: [-5.75187117e-19 -9.66314356e-19 -6.90224540e-20] [1 1 1]
(5, 5, 5)
f, cell#: [-5.75187117e-19 -9.66314356e-19 -6.90224540e-20] [1 1 1]
(5, 5, 5)
f, cell#: [-5.75187117e-19 -9.66314356e-19 -6.90224540e-20] [1 1 1]
(5, 5, 5)
f, cell#: [-5.75187117e-19 -9.66314356e-19 -6.90224540e-20] [1 1 1]
(5, 5, 5)
f, cell#: [-5.75187117e-19 -9.66314356e-19 -6.90224540e-20] [1 1 1]
(5, 5, 5)
f, cell#: [-5.75187117e-19 -9.66314356e-19 -6.90224540e-20] [1 1 1]
(5, 5, 5)
f, cell#: [-5.75187117e-19 -9.66314356e-19 -6.90224540e-20] [1 1 1]
(5, 5, 5)
f, cell#: [-5.75187117e-19 -9.66314356e-19 -6.90224540e-20] [1 1 1]
(5, 5, 5)
f, cell#: [-5.75187117e-19 -9.66314356e-19 -6.90224540e-20] [1 1 1]
(5, 5, 5)
f, cell#: [-5.75187117e-19 -9.66314356e-19 -6.90224540e-20] [1 1 1]
(5, 5, 5)
f, cell#: [-5.75187117e-19 -9.66314356e-19 -6.90224540e-20] [1 1 1]
(5, 5, 5)
f, cell#: [-5.75187117e-19 -9.66314356e-19 -6.90224540

(5, 5, 5)
f, cell#: [-5.75187117e-19 -9.66314356e-19 -6.90224540e-20] [1 1 1]
(5, 5, 5)
f, cell#: [-5.75187117e-19 -9.66314356e-19 -6.90224540e-20] [1 1 1]
(5, 5, 5)
f, cell#: [-5.75187117e-19 -9.66314356e-19 -6.90224540e-20] [1 1 1]
(5, 5, 5)
f, cell#: [-5.75187117e-19 -9.66314356e-19 -6.90224540e-20] [1 1 1]
step  24
(5, 5, 5)
f, cell#: [-5.75187117e-19 -9.66314356e-19 -6.90224540e-20] [1 1 1]
(5, 5, 5)
f, cell#: [-5.75187117e-19 -9.66314356e-19 -6.90224540e-20] [1 1 1]
(5, 5, 5)
f, cell#: [-5.75187117e-19 -9.66314356e-19 -6.90224540e-20] [1 1 1]
(5, 5, 5)
f, cell#: [-5.75187117e-19 -9.66314356e-19 -6.90224540e-20] [1 1 1]
(5, 5, 5)
f, cell#: [-5.75187117e-19 -9.66314356e-19 -6.90224540e-20] [1 1 1]
(5, 5, 5)
f, cell#: [-5.75187117e-19 -9.66314356e-19 -6.90224540e-20] [1 1 1]
(5, 5, 5)
f, cell#: [-5.75187117e-19 -9.66314356e-19 -6.90224540e-20] [1 1 1]
(5, 5, 5)
f, cell#: [-5.75187117e-19 -9.66314356e-19 -6.90224540e-20] [1 1 1]
(5, 5, 5)
f, cell#: [-5.75187117e-19 -9.66314356e-19 -6

...

(5, 5, 5)
f, cell#: [-5.75187117e-19 -9.66314356e-19 -6.90224540e-20] [1 1 1]
(5, 5, 5)
f, cell#: [-5.75187117e-19 -9.66314356e-19 -6.90224540e-20] [1 1 1]
(5, 5, 5)
f, cell#: [-5.75187117e-19 -9.66314356e-19 -6.90224540e-20] [1 1 1]
(5, 5, 5)
f, cell#: [-5.75187117e-19 -9.66314356e-19 -6.90224540e-20] [1 1 1]
(5, 5, 5)
f, cell#: [-5.75187117e-19 -9.66314356e-19 -6.90224540e-20] [1 1 1]
(5, 5, 5)
f, cell#: [-5.75187117e-19 -9.66314356e-19 -6.90224540e-20] [1 1 1]
(5, 5, 5)
f, cell#: [-5.75187117e-19 -9.66314356e-19 -6.90224540e-20] [1 1 1]
(5, 5, 5)
f, cell#: [-5.75187117e-19 -9.66314356e-19 -6.90224540e-20] [1 1 1]
(5, 5, 5)
f, cell#: [-5.75187117e-19 -9.66314356e-19 -6.90224540e-20] [1 1 1]
(5, 5, 5)
f, cell#: [-5.75187117e-19 -9.66314356e-19 -6.90224540e-20] [1 1 1]
(5, 5, 5)
f, cell#: [-5.75187117e-19 -9.66314356e-19 -6.90224540e-20] [1 1 1]
(5, 5, 5)
f, cell#: [-5.75187117e-19 -9.66314356e-19 -6.90224540e-20] [1 1 1]
(5, 5, 5)
f, cell#: [-5.75187117e-19 -9.66314356e-19 -6.90224540

(5, 5, 5)
f, cell#: [-5.75187117e-19 -9.66314356e-19 -6.90224540e-20] [1 1 1]
(5, 5, 5)
f, cell#: [-5.75187117e-19 -9.66314356e-19 -6.90224540e-20] [1 1 1]
(5, 5, 5)
f, cell#: [-5.75187117e-19 -9.66314356e-19 -6.90224540e-20] [1 1 1]
(5, 5, 5)
f, cell#: [-5.75187117e-19 -9.66314356e-19 -6.90224540e-20] [1 1 1]
(5, 5, 5)
f, cell#: [-5.75187117e-19 -9.66314356e-19 -6.90224540e-20] [1 1 1]
(5, 5, 5)
f, cell#: [-5.75187117e-19 -9.66314356e-19 -6.90224540e-20] [1 1 1]
(5, 5, 5)
f, cell#: [-5.75187117e-19 -9.66314356e-19 -6.90224540e-20] [1 1 1]
(5, 5, 5)
f, cell#: [-5.75187117e-19 -9.66314356e-19 -6.90224540e-20] [1 1 1]
(5, 5, 5)
f, cell#: [-5.75187117e-19 -9.66314356e-19 -6.90224540e-20] [1 1 1]
(5, 5, 5)
f, cell#: [-5.75187117e-19 -9.66314356e-19 -6.90224540e-20] [1 1 1]
(5, 5, 5)
f, cell#: [-5.75187117e-19 -9.66314356e-19 -6.90224540e-20] [1 1 1]
(5, 5, 5)
f, cell#: [-5.75187117e-19 -9.66314356e-19 -6.90224540e-20] [1 1 1]
(5, 5, 5)
f, cell#: [-5.75187117e-19 -9.66314356e-19 -6.90224540

(5, 5, 5)
f, cell#: [-5.75187117e-19 -9.66314356e-19 -6.90224540e-20] [1 1 1]
(5, 5, 5)
f, cell#: [-5.75187117e-19 -9.66314356e-19 -6.90224540e-20] [1 1 1]
(5, 5, 5)
f, cell#: [-5.75187117e-19 -9.66314356e-19 -6.90224540e-20] [1 1 1]
(5, 5, 5)
f, cell#: [-5.75187117e-19 -9.66314356e-19 -6.90224540e-20] [1 1 1]
(5, 5, 5)
f, cell#: [-5.75187117e-19 -9.66314356e-19 -6.90224540e-20] [1 1 1]
(5, 5, 5)
f, cell#: [-5.75187117e-19 -9.66314356e-19 -6.90224540e-20] [1 1 1]
(5, 5, 5)
f, cell#: [-5.75187117e-19 -9.66314356e-19 -6.90224540e-20] [1 1 1]
(5, 5, 5)
f, cell#: [-5.75187117e-19 -9.66314356e-19 -6.90224540e-20] [1 1 1]
(5, 5, 5)
f, cell#: [-5.75187117e-19 -9.66314356e-19 -6.90224540e-20] [1 1 1]
(5, 5, 5)
f, cell#: [-5.75187117e-19 -9.66314356e-19 -6.90224540e-20] [1 1 1]
(5, 5, 5)
f, cell#: [-5.75187117e-19 -9.66314356e-19 -6.90224540e-20] [1 1 1]
(5, 5, 5)
f, cell#: [-5.75187117e-19 -9.66314356e-19 -6.90224540e-20] [1 1 1]
(5, 5, 5)
f, cell#: [-5.75187117e-19 -9.66314356e-19 -6.90224540

(5, 5, 5)
f, cell#: [-5.75187117e-19 -9.66314356e-19 -6.90224540e-20] [1 1 1]
(5, 5, 5)
f, cell#: [-5.75187117e-19 -9.66314356e-19 -6.90224540e-20] [1 1 1]
(5, 5, 5)
f, cell#: [-5.75187117e-19 -9.66314356e-19 -6.90224540e-20] [1 1 1]
(5, 5, 5)
f, cell#: [-5.75187117e-19 -9.66314356e-19 -6.90224540e-20] [1 1 1]
(5, 5, 5)
f, cell#: [-5.75187117e-19 -9.66314356e-19 -6.90224540e-20] [1 1 1]
(5, 5, 5)
f, cell#: [-5.75187117e-19 -9.66314356e-19 -6.90224540e-20] [1 1 1]
(5, 5, 5)
f, cell#: [-5.75187117e-19 -9.66314356e-19 -6.90224540e-20] [1 1 1]
(5, 5, 5)
f, cell#: [-5.75187117e-19 -9.66314356e-19 -6.90224540e-20] [1 1 1]
(5, 5, 5)
f, cell#: [-5.75187117e-19 -9.66314356e-19 -6.90224540e-20] [1 1 1]
(5, 5, 5)
f, cell#: [-5.75187117e-19 -9.66314356e-19 -6.90224540e-20] [1 1 1]
(5, 5, 5)
f, cell#: [-5.75187117e-19 -9.66314356e-19 -6.90224540e-20] [1 1 1]
(5, 5, 5)
f, cell#: [-5.75187117e-19 -9.66314356e-19 -6.90224540e-20] [1 1 1]
(5, 5, 5)
f, cell#: [-5.75187117e-19 -9.66314356e-19 -6.90224540

KeyboardInterrupt: 

### Part 1: 
Using this code, show that a single particle starting at rest remains motionless. 

In [198]:
n=1
#single particle at rest
x=np.abs(np.random.randn(n,3))
v=np.random.randn(n,3)*0

print(x, v)

m=np.ones(n) #mass

gridSize = [5, 5, 5]
dt_t = 0.5

parts=Particles(x,v,m, gridSize, grid_dx=dt_t)

dt=0.01

colors = cm.rainbow(np.linspace(0, 1, n))

fig = plt.figure()
ax = Axes3D(fig)    


for k in range(1000):
    for l in range(20):
#         parts.take_step_leapfrog(dt)
        parts.take_step_leapfrog_conv(dt)
        
#     print('step ',k)
    
#     plt.clf() #clears prev graphs
    for n_i in range(n):
        ax.scatter(parts.x[n_i][0], parts.x[n_i][1], parts.x[n_i][2], color=colors[n_i])
        
    ax.set_title("frame {}".format(k))
    plt.pause(0.01)


[[0.37540776 0.30623823 0.37107567]] [[-0.  0. -0.]]
(10, 10, 10)
f, cell#: [2.32688043e-18 5.81720106e-19 8.72580160e-19] [0 0 0]
(10, 10, 10)
f, cell#: [2.32688043e-18 5.81720106e-19 8.72580160e-19] [0 0 0]
(10, 10, 10)
f, cell#: [2.32688043e-18 5.81720106e-19 8.72580160e-19] [0 0 0]
(10, 10, 10)
f, cell#: [2.32688043e-18 5.81720106e-19 8.72580160e-19] [0 0 0]
(10, 10, 10)
f, cell#: [2.32688043e-18 5.81720106e-19 8.72580160e-19] [0 0 0]
(10, 10, 10)
f, cell#: [2.32688043e-18 5.81720106e-19 8.72580160e-19] [0 0 0]
(10, 10, 10)
f, cell#: [2.32688043e-18 5.81720106e-19 8.72580160e-19] [0 0 0]
(10, 10, 10)
f, cell#: [2.32688043e-18 5.81720106e-19 8.72580160e-19] [0 0 0]
(10, 10, 10)
f, cell#: [2.32688043e-18 5.81720106e-19 8.72580160e-19] [0 0 0]
(10, 10, 10)
f, cell#: [2.32688043e-18 5.81720106e-19 8.72580160e-19] [0 0 0]
(10, 10, 10)
f, cell#: [2.32688043e-18 5.81720106e-19 8.72580160e-19] [0 0 0]
(10, 10, 10)
f, cell#: [2.32688043e-18 5.81720106e-19 8.72580160e-19] [0 0 0]
(10, 10, 10

(10, 10, 10)
f, cell#: [2.32688043e-18 5.81720106e-19 8.72580160e-19] [0 0 0]
(10, 10, 10)
f, cell#: [2.32688043e-18 5.81720106e-19 8.72580160e-19] [0 0 0]
(10, 10, 10)
f, cell#: [2.32688043e-18 5.81720106e-19 8.72580160e-19] [0 0 0]
(10, 10, 10)
f, cell#: [2.32688043e-18 5.81720106e-19 8.72580160e-19] [0 0 0]
(10, 10, 10)
f, cell#: [2.32688043e-18 5.81720106e-19 8.72580160e-19] [0 0 0]
(10, 10, 10)
f, cell#: [2.32688043e-18 5.81720106e-19 8.72580160e-19] [0 0 0]
(10, 10, 10)
f, cell#: [2.32688043e-18 5.81720106e-19 8.72580160e-19] [0 0 0]
(10, 10, 10)
f, cell#: [2.32688043e-18 5.81720106e-19 8.72580160e-19] [0 0 0]
(10, 10, 10)
f, cell#: [2.32688043e-18 5.81720106e-19 8.72580160e-19] [0 0 0]
(10, 10, 10)
f, cell#: [2.32688043e-18 5.81720106e-19 8.72580160e-19] [0 0 0]
(10, 10, 10)
f, cell#: [2.32688043e-18 5.81720106e-19 8.72580160e-19] [0 0 0]
(10, 10, 10)
f, cell#: [2.32688043e-18 5.81720106e-19 8.72580160e-19] [0 0 0]
(10, 10, 10)
f, cell#: [2.32688043e-18 5.81720106e-19 8.72580160

(10, 10, 10)
f, cell#: [2.32688043e-18 5.81720106e-19 8.72580160e-19] [0 0 0]
(10, 10, 10)
f, cell#: [2.32688043e-18 5.81720106e-19 8.72580160e-19] [0 0 0]
(10, 10, 10)
f, cell#: [2.32688043e-18 5.81720106e-19 8.72580160e-19] [0 0 0]
(10, 10, 10)
f, cell#: [2.32688043e-18 5.81720106e-19 8.72580160e-19] [0 0 0]
(10, 10, 10)
f, cell#: [2.32688043e-18 5.81720106e-19 8.72580160e-19] [0 0 0]
(10, 10, 10)
f, cell#: [2.32688043e-18 5.81720106e-19 8.72580160e-19] [0 0 0]
(10, 10, 10)
f, cell#: [2.32688043e-18 5.81720106e-19 8.72580160e-19] [0 0 0]
(10, 10, 10)
f, cell#: [2.32688043e-18 5.81720106e-19 8.72580160e-19] [0 0 0]
(10, 10, 10)
f, cell#: [2.32688043e-18 5.81720106e-19 8.72580160e-19] [0 0 0]
(10, 10, 10)
f, cell#: [2.32688043e-18 5.81720106e-19 8.72580160e-19] [0 0 0]
(10, 10, 10)
f, cell#: [2.32688043e-18 5.81720106e-19 8.72580160e-19] [0 0 0]
(10, 10, 10)
f, cell#: [2.32688043e-18 5.81720106e-19 8.72580160e-19] [0 0 0]
(10, 10, 10)
f, cell#: [2.32688043e-18 5.81720106e-19 8.72580160

(10, 10, 10)
f, cell#: [2.32688043e-18 5.81720106e-19 8.72580160e-19] [0 0 0]
(10, 10, 10)
f, cell#: [2.32688043e-18 5.81720106e-19 8.72580160e-19] [0 0 0]
(10, 10, 10)
f, cell#: [2.32688043e-18 5.81720106e-19 8.72580160e-19] [0 0 0]
(10, 10, 10)
f, cell#: [2.32688043e-18 5.81720106e-19 8.72580160e-19] [0 0 0]
(10, 10, 10)
f, cell#: [2.32688043e-18 5.81720106e-19 8.72580160e-19] [0 0 0]
(10, 10, 10)
f, cell#: [2.32688043e-18 5.81720106e-19 8.72580160e-19] [0 0 0]
(10, 10, 10)
f, cell#: [2.32688043e-18 5.81720106e-19 8.72580160e-19] [0 0 0]
(10, 10, 10)
f, cell#: [2.32688043e-18 5.81720106e-19 8.72580160e-19] [0 0 0]
(10, 10, 10)
f, cell#: [2.32688043e-18 5.81720106e-19 8.72580160e-19] [0 0 0]
(10, 10, 10)
f, cell#: [2.32688043e-18 5.81720106e-19 8.72580160e-19] [0 0 0]
(10, 10, 10)
f, cell#: [2.32688043e-18 5.81720106e-19 8.72580160e-19] [0 0 0]
(10, 10, 10)
f, cell#: [2.32688043e-18 5.81720106e-19 8.72580160e-19] [0 0 0]
(10, 10, 10)
f, cell#: [2.32688043e-18 5.81720106e-19 8.72580160

(10, 10, 10)
f, cell#: [2.32688043e-18 5.81720106e-19 8.72580160e-19] [0 0 0]
(10, 10, 10)
f, cell#: [2.32688043e-18 5.81720106e-19 8.72580160e-19] [0 0 0]
(10, 10, 10)
f, cell#: [2.32688043e-18 5.81720106e-19 8.72580160e-19] [0 0 0]
(10, 10, 10)
f, cell#: [2.32688043e-18 5.81720106e-19 8.72580160e-19] [0 0 0]
(10, 10, 10)
f, cell#: [2.32688043e-18 5.81720106e-19 8.72580160e-19] [0 0 0]
(10, 10, 10)
f, cell#: [2.32688043e-18 5.81720106e-19 8.72580160e-19] [0 0 0]
(10, 10, 10)
f, cell#: [2.32688043e-18 5.81720106e-19 8.72580160e-19] [0 0 0]
(10, 10, 10)
f, cell#: [2.32688043e-18 5.81720106e-19 8.72580160e-19] [0 0 0]
(10, 10, 10)
f, cell#: [2.32688043e-18 5.81720106e-19 8.72580160e-19] [0 0 0]
(10, 10, 10)
f, cell#: [2.32688043e-18 5.81720106e-19 8.72580160e-19] [0 0 0]
(10, 10, 10)
f, cell#: [2.32688043e-18 5.81720106e-19 8.72580160e-19] [0 0 0]
(10, 10, 10)
f, cell#: [2.32688043e-18 5.81720106e-19 8.72580160e-19] [0 0 0]
(10, 10, 10)
f, cell#: [2.32688043e-18 5.81720106e-19 8.72580160

(10, 10, 10)
f, cell#: [2.32688043e-18 5.81720106e-19 8.72580160e-19] [0 0 0]
(10, 10, 10)
f, cell#: [2.32688043e-18 5.81720106e-19 8.72580160e-19] [0 0 0]
(10, 10, 10)
f, cell#: [2.32688043e-18 5.81720106e-19 8.72580160e-19] [0 0 0]
(10, 10, 10)
f, cell#: [2.32688043e-18 5.81720106e-19 8.72580160e-19] [0 0 0]
(10, 10, 10)
f, cell#: [2.32688043e-18 5.81720106e-19 8.72580160e-19] [0 0 0]
(10, 10, 10)
f, cell#: [2.32688043e-18 5.81720106e-19 8.72580160e-19] [0 0 0]
(10, 10, 10)
f, cell#: [2.32688043e-18 5.81720106e-19 8.72580160e-19] [0 0 0]
(10, 10, 10)
f, cell#: [2.32688043e-18 5.81720106e-19 8.72580160e-19] [0 0 0]
(10, 10, 10)
f, cell#: [2.32688043e-18 5.81720106e-19 8.72580160e-19] [0 0 0]
(10, 10, 10)
f, cell#: [2.32688043e-18 5.81720106e-19 8.72580160e-19] [0 0 0]
(10, 10, 10)
f, cell#: [2.32688043e-18 5.81720106e-19 8.72580160e-19] [0 0 0]
(10, 10, 10)
f, cell#: [2.32688043e-18 5.81720106e-19 8.72580160e-19] [0 0 0]
(10, 10, 10)
f, cell#: [2.32688043e-18 5.81720106e-19 8.72580160

(10, 10, 10)
f, cell#: [2.32688043e-18 5.81720106e-19 8.72580160e-19] [0 0 0]
(10, 10, 10)
f, cell#: [2.32688043e-18 5.81720106e-19 8.72580160e-19] [0 0 0]
(10, 10, 10)
f, cell#: [2.32688043e-18 5.81720106e-19 8.72580160e-19] [0 0 0]
(10, 10, 10)
f, cell#: [2.32688043e-18 5.81720106e-19 8.72580160e-19] [0 0 0]
(10, 10, 10)
f, cell#: [2.32688043e-18 5.81720106e-19 8.72580160e-19] [0 0 0]
(10, 10, 10)
f, cell#: [2.32688043e-18 5.81720106e-19 8.72580160e-19] [0 0 0]
(10, 10, 10)
f, cell#: [2.32688043e-18 5.81720106e-19 8.72580160e-19] [0 0 0]
(10, 10, 10)
f, cell#: [2.32688043e-18 5.81720106e-19 8.72580160e-19] [0 0 0]
(10, 10, 10)
f, cell#: [2.32688043e-18 5.81720106e-19 8.72580160e-19] [0 0 0]
(10, 10, 10)
f, cell#: [2.32688043e-18 5.81720106e-19 8.72580160e-19] [0 0 0]
(10, 10, 10)
f, cell#: [2.32688043e-18 5.81720106e-19 8.72580160e-19] [0 0 0]
(10, 10, 10)
f, cell#: [2.32688043e-18 5.81720106e-19 8.72580160e-19] [0 0 0]
(10, 10, 10)
f, cell#: [2.32688043e-18 5.81720106e-19 8.72580160

KeyboardInterrupt: 

### Part 2: 
Next, show that a pair of particles placed in a circular orbit continue to orbit each other, for at least some reasonable length of time. 

In [197]:
n=2
#two particles in circular orbit

gridSize = [20, 20, 20]

x=np.zeros([n,3])
v=np.zeros([n,3])
x[0,0]=gridSize[0]/2
x[0,1]=gridSize[1]/2
x[0,2]=gridSize[2]/2 - 1

x[1,0]=gridSize[0]/2
x[1,1]=gridSize[1]/2
x[1,2]=gridSize[2]/2 + 1

# v[0,1]=0.25
# v[1,1]=-0.25
# v[0,2]=0.25
# v[1,2]=-0.25

v[0,0]=0.1
v[0,1]=0.1

v[1,0]=-.1
v[1,1]=-.1

m=np.ones(n) #mass

dt_t = 1.0#1.0
parts=Particles(x,v,m, gridSize, grid_dx=dt_t)

dt=0.05

colors = cm.rainbow(np.linspace(0, 1, n)) 

fig = plt.figure()
ax = Axes3D(fig)
        
for k in range(100):
    for l in range(50):
        parts.take_step_leapfrog_conv(dt)
    
    print('step ',k)

    #     plt.clf() #clears prev graphs
    for n_i in range(n):
        if k==0:
            ax.scatter(parts.x[n_i][0], parts.x[n_i][1], parts.x[n_i][2])#, color=colors[n_i])
            
        else:
            ax.scatter(parts.x[n_i][0], parts.x[n_i][1], parts.x[n_i][2], color=colors[n_i])
#     ax.set_title("frame {}".format(k))
#     ax.text2D(0.05, 0.95, "frame {}".format(k), transform=ax.transAxes)
    plt.pause(0.01)

(20, 20, 20)
f, cell#: [-5.75187117e-20 -6.32705829e-20 -2.24443264e-06] [10 10  9]
f, cell#: [ 7.84346068e-21 -1.25495371e-20  6.12117993e-07] [10 10 11]
(20, 20, 20)
f, cell#: [ 7.12941992e-07  7.12941992e-07 -7.22175550e-07] [10 10  8]
f, cell#: [-7.12941992e-07 -7.12941992e-07  7.22175550e-07] [ 9  9 11]
(20, 20, 20)
f, cell#: [ 7.12941992e-07  7.12941992e-07 -7.22175550e-07] [10 10  8]
f, cell#: [-7.12941992e-07 -7.12941992e-07  7.22175550e-07] [ 9  9 11]
(20, 20, 20)
f, cell#: [ 7.12941992e-07  7.12941992e-07 -7.22175550e-07] [10 10  8]
f, cell#: [-7.12941992e-07 -7.12941992e-07  7.22175550e-07] [ 9  9 11]
(20, 20, 20)
f, cell#: [ 7.12941992e-07  7.12941992e-07 -7.22175550e-07] [10 10  8]
f, cell#: [-7.12941992e-07 -7.12941992e-07  7.22175550e-07] [ 9  9 11]
(20, 20, 20)
f, cell#: [ 7.12941992e-07  7.12941992e-07 -7.22175550e-07] [10 10  8]
f, cell#: [-7.12941992e-07 -7.12941992e-07  7.22175550e-07] [ 9  9 11]
(20, 20, 20)
f, cell#: [ 7.12941992e-07  7.12941992e-07 -7.22175550e-0

f, cell#: [-7.12941992e-07 -7.12941992e-07  7.22175550e-07] [ 9  9 11]
(20, 20, 20)
f, cell#: [ 7.12941992e-07  7.12941992e-07 -7.22175550e-07] [10 10  8]
f, cell#: [-7.12941992e-07 -7.12941992e-07  7.22175550e-07] [ 9  9 11]
(20, 20, 20)
f, cell#: [ 7.12941992e-07  7.12941992e-07 -7.22175550e-07] [10 10  8]
f, cell#: [-7.12941992e-07 -7.12941992e-07  7.22175550e-07] [ 9  9 11]
step  1
(20, 20, 20)
f, cell#: [ 7.12941992e-07  7.12941992e-07 -7.22175550e-07] [10 10  8]
f, cell#: [-7.12941992e-07 -7.12941992e-07  7.22175550e-07] [ 9  9 11]
(20, 20, 20)
f, cell#: [ 7.12941992e-07  7.12941992e-07 -7.22175550e-07] [10 10  8]
f, cell#: [-7.12941992e-07 -7.12941992e-07  7.22175550e-07] [ 9  9 11]
(20, 20, 20)
f, cell#: [ 7.12941992e-07  7.12941992e-07 -7.22175550e-07] [10 10  8]
f, cell#: [-7.12941992e-07 -7.12941992e-07  7.22175550e-07] [ 9  9 11]
(20, 20, 20)
f, cell#: [ 7.12941992e-07  7.12941992e-07 -7.22175550e-07] [10 10  8]
f, cell#: [-7.12941992e-07 -7.12941992e-07  7.22175550e-07] [ 

(20, 20, 20)
f, cell#: [ 6.07157584e-07  6.07157584e-07 -6.07157584e-07] [11 11  8]
f, cell#: [-6.07157584e-07 -6.07157584e-07  6.07157584e-07] [ 8  8 11]
(20, 20, 20)
f, cell#: [ 6.07157584e-07  6.07157584e-07 -6.07157584e-07] [11 11  8]
f, cell#: [-6.07157584e-07 -6.07157584e-07  6.07157584e-07] [ 8  8 11]
(20, 20, 20)
f, cell#: [ 6.07157584e-07  6.07157584e-07 -6.07157584e-07] [11 11  8]
f, cell#: [-6.07157584e-07 -6.07157584e-07  6.07157584e-07] [ 8  8 11]
(20, 20, 20)
f, cell#: [ 6.07157584e-07  6.07157584e-07 -6.07157584e-07] [11 11  8]
f, cell#: [-6.07157584e-07 -6.07157584e-07  6.07157584e-07] [ 8  8 11]
(20, 20, 20)
f, cell#: [ 6.07157584e-07  6.07157584e-07 -6.07157584e-07] [11 11  8]
f, cell#: [-6.07157584e-07 -6.07157584e-07  6.07157584e-07] [ 8  8 11]
(20, 20, 20)
f, cell#: [ 6.07157584e-07  6.07157584e-07 -6.07157584e-07] [11 11  8]
f, cell#: [-6.07157584e-07 -6.07157584e-07  6.07157584e-07] [ 8  8 11]
(20, 20, 20)
f, cell#: [ 6.07157584e-07  6.07157584e-07 -6.07157584e-0

(20, 20, 20)
f, cell#: [ 6.07157584e-07  6.07157584e-07 -6.07157584e-07] [11 11  8]
f, cell#: [-6.07157584e-07 -6.07157584e-07  6.07157584e-07] [ 8  8 11]
(20, 20, 20)
f, cell#: [ 6.07157584e-07  6.07157584e-07 -6.07157584e-07] [11 11  8]
f, cell#: [-6.07157584e-07 -6.07157584e-07  6.07157584e-07] [ 8  8 11]
(20, 20, 20)
f, cell#: [ 6.07157584e-07  6.07157584e-07 -6.07157584e-07] [11 11  8]
f, cell#: [-6.07157584e-07 -6.07157584e-07  6.07157584e-07] [ 8  8 11]
(20, 20, 20)
f, cell#: [ 6.07157584e-07  6.07157584e-07 -6.07157584e-07] [11 11  8]
f, cell#: [-6.07157584e-07 -6.07157584e-07  6.07157584e-07] [ 8  8 11]
(20, 20, 20)
f, cell#: [ 6.07157584e-07  6.07157584e-07 -6.07157584e-07] [11 11  8]
f, cell#: [-6.07157584e-07 -6.07157584e-07  6.07157584e-07] [ 8  8 11]
(20, 20, 20)
f, cell#: [ 6.07157584e-07  6.07157584e-07 -6.07157584e-07] [11 11  8]
f, cell#: [-6.07157584e-07 -6.07157584e-07  6.07157584e-07] [ 8  8 11]
(20, 20, 20)
f, cell#: [ 6.07157584e-07  6.07157584e-07 -6.07157584e-0

(20, 20, 20)
f, cell#: [ 6.07157584e-07  6.07157584e-07 -6.07157584e-07] [11 11  8]
f, cell#: [-6.07157584e-07 -6.07157584e-07  6.07157584e-07] [ 8  8 11]
(20, 20, 20)
f, cell#: [ 6.07157584e-07  6.07157584e-07 -6.07157584e-07] [11 11  8]
f, cell#: [-6.07157584e-07 -6.07157584e-07  6.07157584e-07] [ 8  8 11]
(20, 20, 20)
f, cell#: [ 6.07157584e-07  6.07157584e-07 -6.07157584e-07] [11 11  8]
f, cell#: [-6.07157584e-07 -6.07157584e-07  6.07157584e-07] [ 8  8 11]
(20, 20, 20)
f, cell#: [ 6.07157584e-07  6.07157584e-07 -6.07157584e-07] [11 11  8]
f, cell#: [-6.07157584e-07 -6.07157584e-07  6.07157584e-07] [ 8  8 11]
(20, 20, 20)
f, cell#: [ 6.07157584e-07  6.07157584e-07 -6.07157584e-07] [11 11  8]
f, cell#: [-6.07157584e-07 -6.07157584e-07  6.07157584e-07] [ 8  8 11]
(20, 20, 20)
f, cell#: [ 6.07157584e-07  6.07157584e-07 -6.07157584e-07] [11 11  8]
f, cell#: [-6.07157584e-07 -6.07157584e-07  6.07157584e-07] [ 8  8 11]
(20, 20, 20)
f, cell#: [ 6.07157584e-07  6.07157584e-07 -6.07157584e-0

...

(20, 20, 20)
f, cell#: [-2.79271802e-08 -2.79271802e-08 -2.79271802e-08] [18 18  8]
f, cell#: [2.79271802e-08 2.79271802e-08 2.79271802e-08] [ 1  1 11]
(20, 20, 20)
f, cell#: [-2.79271802e-08 -2.79271802e-08 -2.79271802e-08] [18 18  8]
f, cell#: [2.79271802e-08 2.79271802e-08 2.79271802e-08] [ 1  1 11]
(20, 20, 20)
f, cell#: [-2.79271802e-08 -2.79271802e-08 -2.79271802e-08] [18 18  8]
f, cell#: [2.79271802e-08 2.79271802e-08 2.79271802e-08] [ 1  1 11]
(20, 20, 20)
f, cell#: [-2.79271802e-08 -2.79271802e-08 -2.79271802e-08] [18 18  8]
f, cell#: [2.79271802e-08 2.79271802e-08 2.79271802e-08] [ 1  1 11]
(20, 20, 20)
f, cell#: [-2.79271802e-08 -2.79271802e-08 -2.79271802e-08] [18 18  8]
f, cell#: [2.79271802e-08 2.79271802e-08 2.79271802e-08] [ 1  1 11]
(20, 20, 20)
f, cell#: [-2.79271802e-08 -2.79271802e-08 -2.79271802e-08] [18 18  8]
f, cell#: [2.79271802e-08 2.79271802e-08 2.79271802e-08] [ 1  1 11]
(20, 20, 20)
f, cell#: [-2.79271802e-08 -2.79271802e-08 -2.79271802e-08] [18 18  8]
f, c

(20, 20, 20)
f, cell#: [-1.07282653e-08 -1.07282653e-08 -1.08672107e-08] [19 19  8]
f, cell#: [1.07282653e-08 1.07282653e-08 1.08672107e-08] [ 0  0 11]
(20, 20, 20)
f, cell#: [-1.07282653e-08 -1.07282653e-08 -1.08672107e-08] [19 19  8]
f, cell#: [1.07282653e-08 1.07282653e-08 1.08672107e-08] [ 0  0 11]
(20, 20, 20)
f, cell#: [-1.07282653e-08 -1.07282653e-08 -1.08672107e-08] [19 19  8]
f, cell#: [1.07282653e-08 1.07282653e-08 1.08672107e-08] [ 0  0 11]
(20, 20, 20)
f, cell#: [-1.07282653e-08 -1.07282653e-08 -1.08672107e-08] [19 19  8]
f, cell#: [1.07282653e-08 1.07282653e-08 1.08672107e-08] [ 0  0 11]
(20, 20, 20)
f, cell#: [-1.07282653e-08 -1.07282653e-08 -1.08672107e-08] [19 19  8]
f, cell#: [1.07282653e-08 1.07282653e-08 1.08672107e-08] [ 0  0 11]
(20, 20, 20)
f, cell#: [-1.07282653e-08 -1.07282653e-08 -1.08672107e-08] [19 19  8]
f, cell#: [1.07282653e-08 1.07282653e-08 1.08672107e-08] [ 0  0 11]
(20, 20, 20)
f, cell#: [-1.07282653e-08 -1.07282653e-08 -1.08672107e-08] [19 19  8]
f, c

(20, 20, 20)
f, cell#: [-1.07282653e-08 -1.07282653e-08 -1.08672107e-08] [19 19  8]
f, cell#: [1.07282653e-08 1.07282653e-08 1.08672107e-08] [ 0  0 11]
(20, 20, 20)
f, cell#: [-1.07282653e-08 -1.07282653e-08 -1.08672107e-08] [19 19  8]
f, cell#: [1.07282653e-08 1.07282653e-08 1.08672107e-08] [ 0  0 11]
(20, 20, 20)
f, cell#: [-1.07282653e-08 -1.07282653e-08 -1.08672107e-08] [19 19  8]
f, cell#: [1.07282653e-08 1.07282653e-08 1.08672107e-08] [ 0  0 11]
(20, 20, 20)
f, cell#: [-1.07282653e-08 -1.07282653e-08 -1.08672107e-08] [19 19  8]
f, cell#: [1.07282653e-08 1.07282653e-08 1.08672107e-08] [ 0  0 11]
(20, 20, 20)
f, cell#: [-1.07282653e-08 -1.07282653e-08 -1.08672107e-08] [19 19  8]
f, cell#: [1.07282653e-08 1.07282653e-08 1.08672107e-08] [ 0  0 11]
(20, 20, 20)
f, cell#: [-1.07282653e-08 -1.07282653e-08 -1.08672107e-08] [19 19  8]
f, cell#: [1.07282653e-08 1.07282653e-08 1.08672107e-08] [ 0  0 11]
(20, 20, 20)
f, cell#: [-1.07282653e-08 -1.07282653e-08 -1.08672107e-08] [19 19  8]
f, c

(20, 20, 20)


IndexError: index 20 is out of bounds for axis 0 with size 20

### Part 3: 
Set up both periodic and non-periodic boundary conditions. Set up a problem where hundreds of thousands of particles are initially scattered randomly throughout the domain. Show the evolution with time for both periodic and non-periodic boundary conditions. Track the total energy how well is it conserved? 

### Part 4: 
In cosmology, we start the universe with a scale-invariant power spectrum, so mass fluctuations are proportional to k−3. Start with the particles on a grid, but with masses derived from a realization of k−3 and use periodic boundary conditions (although not strictly necessary, you may with to start with your particles in the center of grid cells rather than at the corners). How does your universe look now?